In [1]:
import warnings
warnings.simplefilter('ignore')

import os
import gc
import re
import glob

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
from tqdm.auto import tqdm

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, auc

from urllib.parse import quote, unquote, urlparse

import lightgbm as lgb

In [2]:
# train

train_files = glob.glob('data/train/*.csv')

df_train = pd.DataFrame()

for filepath in tqdm(train_files):
    df = pd.read_csv(filepath)
    df_train = pd.concat([df_train, df]).reset_index(drop=True)
    
df_train.fillna('__NaN__', inplace=True)

# 强迫症发作..
df_train = df_train.rename(columns={'lable': 'label'})
df_train

  0%|          | 0/6 [00:00<?, ?it/s]

,id,method,user_agent,url,refer,body,label
0,2254,PUT,Mozilla/5.0 (Windows NT 10.0; rv:78.0) Gecko/2...,/vlxmgs.jsp/,__NaN__,yzhcrcncpppk <%-- kdsohssjxbpn --%>,3
1,10721,GET,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,/manage/,__NaN__,GET /manage/ HTTP/1.1 Host: dining.ccb.com Upg...,3
2,990,POST,Mozilla/5.0 (Windows NT 10.0; rv:78.0) Gecko/2...,/host-manager/?q=user/password&name[%23post_re...,__NaN__,form_id=user_pass&_triggering_element_name=nam...,3
3,631,POST,__NaN__,/Object.block/qkauth/MzE4MmZjNzYvMTcwYS80N2E4L...,__NaN__,--BjX22NEYB4KXue8hEFnMUjp0KEOY4xP4 Content-Dis...,3
4,8149,GET,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,/star/media/news/c/c_20211206_5650913.shtml,http://star.sse.com.cn/,GET /star/media/news/c/c_20211206_5650913.shtm...,3
...,...,...,...,...,...,...,...
33214,19942,GET,__NaN__,/cv?uuid=69c0af3ce4caa8a131e86d0a879c55d6&tc=7...,__NaN__,__NaN__,0
33215,11326,GET,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,/c.html?p=mp_set&pid=setting&mid=48db3e8ce61fc...,__NaN__,__NaN__,0
33216,509,GET,__NaN__,/market/seller/ww/biz_config.php,__NaN__,__NaN__,0
33217,4794,GET,Wget/1.11.4,/stats.php?rand=F9VvXjwI,__NaN__,__NaN__,0


In [3]:
# label
# 0. 白
# 1. SQL 注入
# 2. 目录历遍
# 3. 远程代码执行
# 4. 命令执行
# 5. XSS 跨站脚本

In [4]:
df_test = pd.read_csv('data/test/test.csv')
df_test.fillna('__NaN__', inplace=True)
df_test

,id,method,user_agent,url,refer,body
0,0,GET,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,/demo/aisec/upload.php?act='%7C%7C(select+1+fr...,http://demo.aisec.cn/demo/aisec/upload.php?t=0...,GET /demo/aisec/upload.php?act='%7C%7C(select+...
1,1,GET,Dalvik/2.1.0 (Linux; U; Android 11; M2102J2SC ...,/livemsg?ad_type=WL_WK&ty=web&pu=1&openudid=5f...,__NaN__,GET /livemsg?ad_type=WL_WK&ty=web&pu=1&openudi...
2,2,GET,Mozilla/5.0 (Windows NT 10.0; rv:78.0) Gecko/2...,/create_user/?username=%3Cscript%3Ealert(docum...,__NaN__,__NaN__
3,3,GET,__NaN__,/mmsns/WeDwicXmkOl4kjKsBycicI0H3q41r6syFFvu46h...,__NaN__,__NaN__
4,4,PUT,Mozilla/5.0 (Windows NT 10.0; rv:78.0) Gecko/2...,/naizau.jsp/,__NaN__,GET /login HTTP/1.1 Host: 111.160.211.18:8088 ...
...,...,...,...,...,...,...
3995,3995,GET,Dalvik/2.1.0 (Linux; U; Android 10; POT-AL00 B...,/livemsg?ad_type=WL_WK&ty=web&pu=1&openudid=64...,__NaN__,GET /livemsg?ad_type=WL_WK&ty=web&pu=1&openudi...
3996,3996,GET,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,/runtime.js,http://121.4.111.58:3000/,GET /runtime.js HTTP/1.1 Host: 121.4.111.58:30...
3997,3997,POST,Mozilla/4.0,/query?493521812,__NaN__,__NaN__
3998,3998,GET,Wget/1.11.4,/stats.php?rand=JtmT4wBtrpNy5RJnNX9wCUo,__NaN__,__NaN__


In [5]:
df = pd.concat([df_train, df_test]).reset_index(drop=True)
df.shape

(37219, 7)

In [6]:
def get_url_query(s):
    li = re.split('[=&]', urlparse(s)[4])
    return [li[i] for i in range(len(li)) if i % 2 == 1]


def find_max_str_length(x):
    max_ = 0
    li = [len(i) for i in x]
    return max(li) if len(li) > 0 else 0


def find_str_length_std(x):
    max_ = 0
    li = [len(i) for i in x]
    return np.std(li) if len(li) > 0 else -1


df['url_unquote'] = df['url'].apply(unquote)
df['url_query'] = df['url_unquote'].apply(lambda x: get_url_query(x))
df['url_query_num'] = df['url_query'].apply(len)
df['url_query_max_len'] = df['url_query'].apply(find_max_str_length)
df['url_query_len_std'] = df['url_query'].apply(find_str_length_std)

In [7]:
def find_url_filetype(x):
    try:
        return re.search(r'\.[a-z]+', x).group()
    except:
        return '__NaN__'
    
    
df['url_path'] = df['url_unquote'].apply(lambda x: urlparse(x)[2])
df['url_filetype'] = df['url_path'].apply(lambda x: find_url_filetype(x))

df['url_path_len'] = df['url_path'].apply(len)
df['url_path_num'] = df['url_path'].apply(lambda x: len(re.findall('/',  x)))

In [8]:
df['ua_short'] = df['user_agent'].apply(lambda x: x.split('/')[0])
df['ua_first'] = df['user_agent'].apply(lambda x: x.split(' ')[0])

In [9]:
%%time

def add_tfidf_feats(df, col, n_components=16):
    text = list(df[col].values)
    tf = TfidfVectorizer(min_df=1, 
                         analyzer='char_wb',
                         ngram_range=(1,3), 
                         stop_words='english')
    tf.fit(text)
    X = tf.transform(text)
    svd = TruncatedSVD(n_components=n_components)
    svd.fit(X)
    X_svd = svd.transform(X)
    for i in range(n_components):
        df[f'{col}_tfidf_{i}'] = X_svd[:, i]
    return df


df = add_tfidf_feats(df, 'url_unquote', n_components=16)
df = add_tfidf_feats(df, 'user_agent', n_components=16)
df = add_tfidf_feats(df, 'body', n_components=32)

CPU times: user 6min 27s, sys: 47.9 s, total: 7min 14s
Wall time: 5min 51s


In [10]:
for col in tqdm(['method', 'refer', 'url_filetype', 'ua_short', 'ua_first']):
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

  0%|          | 0/5 [00:00<?, ?it/s]

In [11]:
not_use_feats = ['id', 'user_agent', 'url', 'body', 'url_unquote', 'url_query', 'url_path', 'label']
use_features = [col for col in df.columns if col not in not_use_feats]

In [12]:
train = df[df['label'].notna()]
test = df[df['label'].isna()]

train.shape, test.shape

((33219, 82), (4000, 82))

In [13]:
NUM_CLASSES = 6
FOLDS = 5
TARGET = 'label'

from sklearn.preprocessing import label_binarize

def run_lgb(df_train, df_test, use_features):
    
    target = TARGET
    oof_pred = np.zeros((len(df_train), NUM_CLASSES))
    y_pred = np.zeros((len(df_test), NUM_CLASSES))
    
    folds = StratifiedKFold(n_splits=FOLDS)
    for fold, (tr_ind, val_ind) in enumerate(folds.split(train, train[TARGET])):
        print(f'Fold {fold + 1}')
        x_train, x_val = df_train[use_features].iloc[tr_ind], df_train[use_features].iloc[val_ind]
        y_train, y_val = df_train[target].iloc[tr_ind], df_train[target].iloc[val_ind]
        train_set = lgb.Dataset(x_train, y_train)
        val_set = lgb.Dataset(x_val, y_val)
        
        params = {
            'learning_rate': 0.1,
            'metric': 'multiclass',
            'objective': 'multiclass',
            'num_classes': NUM_CLASSES,
            'feature_fraction': 0.75,
            'bagging_fraction': 0.75,
            'bagging_freq': 2,
            'n_jobs': -1,
            'seed': 1029,
            'max_depth': 10,
            'num_leaves': 100,
            'lambda_l1': 0.5,
            'lambda_l2': 0.8,
            'verbose': -1
        }
        
        model = lgb.train(params, 
                          train_set, 
                          num_boost_round=500,
                          early_stopping_rounds=100,
                          valid_sets=[train_set, val_set],
                          verbose_eval=100)
        oof_pred[val_ind] = model.predict(x_val)
        y_pred += model.predict(df_test[use_features]) / folds.n_splits
        
        print("Features importance...")
        gain = model.feature_importance('gain')
        feat_imp = pd.DataFrame({'feature': model.feature_name(), 
                         'split': model.feature_importance('split'), 
                         'gain': 100 * gain / gain.sum()}).sort_values('gain', ascending=False)
        print('Top 50 features:\n', feat_imp.head(50))
        
        del x_train, x_val, y_train, y_val, train_set, val_set
        gc.collect()
        
    return y_pred, oof_pred
    

y_pred, oof_pred = run_lgb(train, test, use_features)

Fold 1
Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.00565193	valid_1's multi_logloss: 0.067217
Early stopping, best iteration is:
[71]	training's multi_logloss: 0.0083728	valid_1's multi_logloss: 0.0657514
Features importance...
Top 50 features:
                  feature  split       gain
42          body_tfidf_0    337  14.274142
11   url_unquote_tfidf_1    731  11.079421
43          body_tfidf_1    247  10.633122
23  url_unquote_tfidf_13    757   6.634695
13   url_unquote_tfidf_3    673   5.691953
30    user_agent_tfidf_4    249   4.663404
26    user_agent_tfidf_0    300   3.922707
17   url_unquote_tfidf_7    767   2.966712
16   url_unquote_tfidf_6    674   2.798574
15   url_unquote_tfidf_5    614   2.450545
21  url_unquote_tfidf_11    677   2.298384
12   url_unquote_tfidf_2    554   2.080702
56         body_tfidf_14    255   1.941898
47          body_tfidf_5    215   1.671437
3      url_query_max_len    326   1.553312
27    user_age

Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.00597365	valid_1's multi_logloss: 0.0614529
Early stopping, best iteration is:
[86]	training's multi_logloss: 0.00696167	valid_1's multi_logloss: 0.0611329
Features importance...
Top 50 features:
                  feature  split       gain
42          body_tfidf_0    390  14.226127
11   url_unquote_tfidf_1    910  11.452608
43          body_tfidf_1    316  10.768586
23  url_unquote_tfidf_13    916   6.397416
13   url_unquote_tfidf_3    816   5.510094
30    user_agent_tfidf_4    258   4.610067
26    user_agent_tfidf_0    366   4.078298
15   url_unquote_tfidf_5    751   2.870557
16   url_unquote_tfidf_6    799   2.381543
21  url_unquote_tfidf_11    842   2.227994
20  url_unquote_tfidf_10    761   2.158855
12   url_unquote_tfidf_2    702   2.109419
17   url_unquote_tfidf_7   1017   1.974487
56         body_tfidf_14    320   1.774696
47          body_tfidf_5    225   1.680044
27    user_agent_tf

In [14]:
accuracy_score(np.argmax(oof_pred, axis=1), df_train['label'])

0.9801920587615521

In [15]:
sub = pd.read_csv('data/submit_example.csv')
sub['predict'] = np.argmax(y_pred, axis=1)
sub

,id,predict
0,0,1
1,1,1
2,2,5
3,3,0
4,4,3
...,...,...
3995,3995,1
3996,3996,1
3997,3997,0
3998,3998,0


In [16]:
sub['predict'].value_counts()

2    899
0    812
1    809
3    661
4    422
5    397
Name: predict, dtype: int64

In [17]:
sub.to_csv('baseline082901.csv', index=False)